In [19]:
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [20]:
import requests

In [21]:
counties = pd.read_csv('data/county_list.csv',encoding= 'unicode_escape')

In [22]:
counties.head(3)

,county
0,"Stewart County, TN"
1,"Robertson County, TN"
2,"Sumner County, TN"


In [23]:
counties.shape

(99, 1)

In [25]:
county_list = counties['county'].tolist()
#Make dataframe to a list
len(county_list)

99

In [26]:
API_KEY = pd.read_csv('api_key_work.txt', sep=" ", header=None).iloc[0,0]

In [27]:
#Pull geographic centers of each county from Google API
def get_google_results(address, api_key=None):

    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    results = requests.get(geocode_url)
    results = results.json()
    
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            "google_place_id": None,
            "type": None,
            "postcode": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "location_type": answer.get('geometry').get('location_type'),
            "place_id": answer.get("place_id"),
            "types": ",".join(answer.get('types')),
            "partial_match" : answer.get('partial_match'),
            "postcode": ",".join([x['long_name'] for x in answer.get('address_components') 
                                  if 'postal_code' in x.get('types')])
        }
        
    # Append some other details:    
    output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    
    return output

In [28]:
#Test!
geocode_result = get_google_results('str,hendersonville, tn', API_KEY)
geocode_result

{'formatted_address': '735 E Main St, Hendersonville, TN 37075, USA',
 'latitude': 36.3268836,
 'longitude': -86.5591469,
 'location_type': 'ROOFTOP',
 'place_id': 'ChIJ1dQmQ-s4ZIgRSqWf0GqNRuM',
 'types': 'establishment,point_of_interest,travel_agency',
 'partial_match': None,
 'postcode': '37075',
 'input_string': 'str,hendersonville, tn',
 'number_of_results': 1,
 'status': 'OK'}

In [29]:
len(county_list)

99

In [34]:
results = []
for county in county_list:
    try:
        geocode_result = get_google_results(county, API_KEY)
    except:
        None
            
    results.append(geocode_result)           

    if len(results) % 5 == 0:
        pd.DataFrame(results).to_csv("{}_bak.csv".format('countyresults'), index = False)

pd.DataFrame(results)

,formatted_address,latitude,longitude,location_type,place_id,types,partial_match,postcode,input_string,number_of_results,status
0,"Stewart County, TN, USA",36.502324,-87.776333,APPROXIMATE,ChIJH_USb_TgeogR_ZAp2Z30_w0,"administrative_area_level_2,political",None,,"Stewart County, TN",1,OK
1,"Robertson County, TN, USA",36.525067,-86.862183,APPROXIMATE,ChIJSZoDIMxVZIgRCM0qa83_1KA,"administrative_area_level_2,political",None,,"Robertson County, TN",1,OK
2,"Sumner County, TN, USA",36.471808,-86.499655,APPROXIMATE,ChIJ_WmAQCgvZIgRZmf7nHhvxGo,"administrative_area_level_2,political",None,,"Sumner County, TN",1,OK
3,"Montgomery County, TN, USA",36.496041,-87.460397,APPROXIMATE,ChIJcXGk-4_RZIgRXDc05xoLPOo,"administrative_area_level_2,political",None,,"Montgomery County, TN",1,OK
4,"Macon County, TN, USA",36.498388,-85.971322,APPROXIMATE,ChIJt3YaEbGKZogRjUlGfmuKDnA,"administrative_area_level_2,political",None,,"Macon County, TN",1,OK
5,"Clay County, TN, USA",36.570177,-85.561210,APPROXIMATE,ChIJORJwEmYHZ4gRIG_C67qdzAY,"administrative_area_level_2,political",None,,"Clay County, TN",1,OK
6,"Pickett County, TN, USA",36.554109,-85.147936,APPROXIMATE,ChIJ7xg1VeWBZ4gRBCWQKLTN5i8,"administrative_area_level_2,political",None,,"Pickett County, TN",1,OK
7,"Sullivan County, TN, USA",36.493315,-82.345189,APPROXIMATE,ChIJkVvjkRuDWogRGTIgypBlSWE,"administrative_area_level_2,political",None,,"Sullivan County, TN",1,OK
8,"Johnson County, TN, USA",36.458567,-81.822436,APPROXIMATE,ChIJ38d5AqpgUIgRa9NdXuPXLfc,"administrative_area_level_2,political",None,,"Johnson County, TN",1,OK
9,"Scott County, TN, USA",36.397621,-84.522219,APPROXIMATE,ChIJs-dtr56gXYgRjY6TDA5aYr4,"administrative_area_level_2,political",None,,"Scott County, TN",1,OK


In [54]:
county_center = pd.DataFrame(results)[['latitude', 'longitude', 'input_string']]

In [55]:
county_center['input_string'] = [x.split(' ')[0] for x in county_center['input_string']]

In [57]:
county_center.columns = ['lat', 'long', 'county']

In [58]:
county_center.head()

,lat,long,county
0,36.502324,-87.776333,Stewart
1,36.525067,-86.862183,Robertson
2,36.471808,-86.499655,Sumner
3,36.496041,-87.460397,Montgomery
4,36.498388,-85.971322,Macon


In [59]:
#county_center.to_csv('county_centers.csv')